In [1]:
import torch
import numpy as np
import util.npose_util as nu
import os
import pathlib
import dgl
from dgl import backend as F
import torch_geometric
from torch.utils.data import random_split, DataLoader, Dataset
from typing import Dict
from torch import Tensor
from dgl import DGLGraph
from torch import nn
from chemical import cos_ideal_NCAC #from RoseTTAFold2
from torch import einsum
torch.cuda.is_available()

True

In [2]:
#npose indexing
# Useful numbers
# N [-1.45837285,  0 , 0]
# CA [0., 0., 0.]
# C [0.55221403, 1.41890368, 0.        ]
# CB [ 0.52892494, -0.77445692, -1.19923854]

N_CA_dist = torch.tensor(1.458/10.0).to('cuda')
C_CA_dist = torch.tensor(1.523/10.0).to('cuda')

if ( hasattr(os, 'ATOM_NAMES') ):
    assert( hasattr(os, 'PDB_ORDER') )

    ATOM_NAMES = os.ATOM_NAMES
    PDB_ORDER = os.PDB_ORDER
else:
    ATOM_NAMES=['N', 'CA', 'CB', 'C', 'O']
    PDB_ORDER = ['N', 'CA', 'C', 'O', 'CB']

_byte_atom_names = []
_atom_names = []
for i, atom_name in enumerate(ATOM_NAMES):
    long_name = " " + atom_name + "       "
    _atom_names.append(long_name[:4])
    _byte_atom_names.append(atom_name.encode())

    globals()[atom_name] = i

R = len(ATOM_NAMES)

if ( "N" not in globals() ):
    N = -1
if ( "C" not in globals() ):
    C = -1
if ( "CB" not in globals() ):
    CB = -1


_pdb_order = []
for name in PDB_ORDER:
    _pdb_order.append( ATOM_NAMES.index(name) )

In [3]:
# data_path_str  = 'data/h4_ca_coords.npz'
# test_limit = 1028
# rr = np.load(data_path_str)
# ca_coords = [rr[f] for f in rr.files][0][:test_limit,:,:3]
# ca_coords.shape

# getting N-Ca, Ca-C vectors to add as typeI features
#apa = apart helices for test/train split
#tog = together helices for test/train split
apa_path_str  = 'data_npose/h4_apa_coords.npz'
tog_path_str  = 'data_npose/h4_tog_coords.npz'

#grab the first 3 atoms which are N,CA,C
test_limit = 1028
rr = np.load(apa_path_str)
coords_apa = [rr[f] for f in rr.files][0][:test_limit,:]

rr = np.load(tog_path_str)
coords_tog = [rr[f] for f in rr.files][0][:test_limit,:]

In [6]:
t1 = coords_tog[0,:,1,:]

In [31]:
def mp_chunk(coords_in : np.array , reduct: int = 10):
    full_chunks = int(coords_in.shape[0]/reduct)
    final_chunk = int(coords_in.shape[0]%reduct)
    #mean across reduction size
    coords_reduct  = np.mean(coords_in[:(full_chunks-1)*reduct].reshape(full_chunks-1,reduct,-1),axis=1)
    #final chunk adds extra non divisible chunks
    if final_chunk>0:
        coords_reduct  = np.vstack((coords_reduct,np.mean(coords_in[-(final_chunk+reduct):],axis=0)))
        
    return coords_reduct

In [28]:
np.mean(t1[-15:].reshape(1,15,-1),axis=1)

array([[ 4.87193333,  9.76133333, 11.27713333]])

In [26]:
np.mean(coords_in[:(full_chunks-1)*reduct].reshape(full_chunks-1,reduct,-1),axis=1)

array([[ 5.4772    ,  4.4943    ,  1.7107    ],
       [14.8149    ,  5.8221    ,  6.2843    ],
       [ 4.0245    , -3.3235    ,  8.016     ],
       [-5.4647    , -3.2477    ,  8.8498    ],
       [ 2.152     ,  2.8142    , 18.3595    ],
       [ 4.87193333,  9.76133333, 11.27713333]])

In [ ]:
def reduced_coordinates(coordinates, reduction):
    
    coord
    